## Sklearn Pipelines for the Modern ML Engineer: 9 Techniques You Can’t Ignore
- https://towardsdatascience.com/sklearn-pipelines-for-the-modern-ml-engineer-9-techniques-you-cant-ignore-637788f05df5

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.6.25</div>
<div style="text-align: right"> Last update: 2023.6.25</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

### Motivation

파이프라인으로 다음 기능을 처리가능  
- X의 숫자 및 범주형 특징을 자동으로 분리합니다.
- 숫자 특징의 결측값을 대입합니다.
- 왜곡된 특징을 로그 변환하고 나머지는 정규화합니다.
- 범주형 특징에서 결측치를 대입하고 원핫 인코딩합니다.
- 대상 배열 Y를 정규화합니다.

### 0. Estimators vs transformers

Sklearn의 transformer는 데이터세의 피처를 입력으로 받고 변환을 한 다음 결과를 리턴하는 클래스 또는 함수입니다.  
이때 fit_transform 및 transform 메서드를 사용한다.

예를 들어 숫자 입력을 받아 정규 분포로 만드는 QuantileTransformer가 있습니다.  
이 함수는 이상값이 있는 피처에 특히 유용합니다.

transformer는 TransformerMixin 베이스 클래스를 상속합니다.

In [2]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import QuantileTransformer

isinstance(QuantileTransformer(), TransformerMixin)

True

반면에 estimator는 일반적으로 데이터셋에 대한 예측을 리턴하는 클래스이다.  
estimator는 종종 regressor 또는 classifier와 같은 단어로 끝나는 이름을 가진다.

estimator는 BaseEstimator 클래스를 상속받는다.

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator

isinstance(LinearRegression(), BaseEstimator)

True

### 1. Vanilla pipeline
바닐라 파이프라인은 하나 이상의 transformer와 하나의 최종 estimator로 구성됩니다. 

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline

numerical_pipeline = make_pipeline(
    StandardScaler(), SimpleImputer(), LinearRegression()
)

numerical_pipeline

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

- 이때 transformer의 순서가 중요함  

만약 범주형 파이프라인도 있다면 다음과 같이 구성함

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Define the categorical pipeline
categorical_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    StandardScaler(),
    LogisticRegression(),
)
categorical_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

- 각 아이템은 steps라고 하는 인자에 추가된다.   
- make_pipeline 함수는 자동으로 각 step의 이름을 지정한다.   

그런데 자동 지정된 이름은 길수도 있다.  
Pipeline 클래스를 사용하면 직접 이름을 지정할 수 있다.

In [6]:
from sklearn.pipeline import Pipeline

numeric_pipeline = Pipeline(
    steps=[
        ("scale", StandardScaler()),
        ("impute", SimpleImputer()),
        ("lr", LinearRegression()),
    ]
)
numeric_pipeline

Pipeline(steps=[('scale', StandardScaler()), ('impute', SimpleImputer()),
                ('lr', LinearRegression())])

- steps 인자는 튜플로 된 리스트를 받아들이며, 각 아이템은 다음과 같다. 
  - step 이름 문자열
  - transformer 또는 estimator 

### 2. A milkshake of transformers

실제 데이터 집합은 숫자형과 범주형 특징이 혼합되어 있는 경우가 많기 때문에 바닐라 트랜스포머를 단독으로 사용하는 경우는 거의 없습니다.  

따라서 다양한 범주의 트랜스포머를 단일 개체로 결합하는 동시에 데이터 집합 X의 어떤 열에 어떤 트랜스포머를 적용해야 하는지 지정할 수 있는 방법이 필요합니다.

이 기능은 ColumnTransformer 클래스에서 우아하게 구현됩니다.  

In [7]:
from sklearn.preprocessing import OrdinalEncoder

nums = ["numeric_1", "numeric_2", "numeric_3"]
cats = ["categorical_1", "categorical_2", "categorical_3"]  

numeric_pipe = make_pipeline(SimpleImputer(), QuantileTransformer())
categorical_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OrdinalEncoder()
)

In [8]:
from sklearn.compose import ColumnTransformer

transformers = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, nums),
        ("categorical", categorical_pipeline, cats)
    ]
)
transformers

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('scale', StandardScaler()),
                                                 ('impute', SimpleImputer()),
                                                 ('lr', LinearRegression())]),
                                 ['numeric_1', 'numeric_2', 'numeric_3']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler()),
                                                 ('logisticregression',
                                                  LogisticRegression())]),
                                 ['categorical_1', 'categorical_2',
                                  'categorical_3'])])

- ColumnTransformer의 transformers 인자는 다음 세가지 아이템으로 된 튜플 리스트를 받는다.
    - The name of the step.
    - The transformer or a pipeline of transformers.
    - The name of the columns to which the transformers should be applied.

### 3. A milkshake with a watermelon on top   
여기에 estimator를 추가해서 파이프라인을 만들어 보자.

In [9]:
full_pipeline_reg = make_pipeline(transformers, LinearRegression())
full_pipeline_reg

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler()),
                                                                  ('impute',
                                                                   SimpleImputer()),
                                                                  ('lr',
                                                                   LinearRegression())]),
                                                  ['numeric_1', 'numeric_2',
                                                   'numeric_3']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('logisticregression',
                                                                   LogisticRegression())]),
                                                  ['categorical_1',
                                                   'categorical_2',
                                                   'categorical_3'])])),
                ('linearregression', LinearRegression())])

In [10]:
# 또다른 방법  
full_pipeline_clf = Pipeline(
    steps=[
        ("preprocess", transformers),
        ("clf", LogisticRegression()),
    ]
)
full_pipeline_clf

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler()),
                                                                  ('impute',
                                                                   SimpleImputer()),
                                                                  ('lr',
                                                                   LinearRegression())]),
                                                  ['numeric_1', 'numeric_2',
                                                   'numeric_3']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('logisticregression',
                                                                   LogisticRegression())]),
                                                  ['categorical_1',
                                                   'categorical_2',
                                                   'categorical_3'])])),
                ('clf', LogisticRegression())])

마지막으로 fit, predict 메서드를 사용하여 모델을 학습, 예측할 수 있다.

```python
# y is a classification label
full_pipeline_clf.fit(X, y)

# y is a numeric label
full_pipeline_reg.fit(X, y)
```


### 4. Choosing columns with style
앞에서는 실수, 범주형 피처를 하나씩 수동으로 지정했음.   
자동으로 지정하는 방법이 있음

In [11]:
import numpy as np
from sklearn.compose import make_column_selector

numeric_cols = make_column_selector(dtype_include=np.number)
categoricals = make_column_selector(dtype_exclude=np.number)

필터링하기 위한 정규표현식 패턴을 저장할 수도 있다.  
만약 `word1`또는 `word2`로 시작하는 컬럼을 선택하고 싶다면 다음과 같이 작성한다.

In [12]:
pattern = "^(word1|word2)"
filtered_columns = make_column_selector(pattern)

이 함수와 ColumnTransformer 를 저합하여 다른 타입의 데이터셋을 전처리 할 수 있다.

In [13]:
from sklearn.compose import make_column_transformer, make_column_selector

nums = make_column_selector(dtype_include = np.number)
cats = make_column_selector(dtype_exclude=np.number)

numeric_pipe = make_pipeline(...)
categorical_pipe = make_pipeline(...)

transformers = make_column_transformer(
    (nums, numeric_pipe), (cats, categorical_pipe)
)

transformers

ColumnTransformer(transformers=[('make_column_selector-1',
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f37231776a0>,
                                 Pipeline(steps=[('ellipsis', Ellipsis)])),
                                ('make_column_selector-2',
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f3723177bb0>,
                                 Pipeline(steps=[('ellipsis', Ellipsis)]))])

### 5. visual pipelines

In [14]:
from sklearn import set_config

set_config(display="diagram")

### 6. Pipeline cache

파이프라인이 준비되면 24시간 연중무휴로 실행하고 싶을 것입니다. 하지만 파이프라인에는 데이터를 조작하는 여러 트랜스포머가 포함되어 있으므로 동일한 작업을 다시 실행하는 데 많은 시간이 소요될 수 있습니다.

이 문제를 해결하기 위해 Sklearn은 파이프라인 내에서 트랜스포머의 출력을 캐시할 수 있는 메모리 인수를 제공합니다. 이 캐싱 메커니즘은 트랜스포머 출력의 불필요한 재계산을 방지하는 데 도움이 됩니다. 사용 방법은 다음과 같습니다:

In [15]:
from shutil import rmtree
from tempfile import mkdtemp

from sklearn.decomposition import PCA

# make temp directory
chache_dir = mkdtemp()

estimators = [("reduce_dim", PCA()), ("clf", LinearRegression())]
my_pipe = Pipeline(estimators, memory=chache_dir)

# run the pipeline
#...

# remove the chache directory at the end of your script
rmtree(chache_dir)

캐싱을 활성화하려면 mkdtemp 함수를 사용하여 임시 디렉터리를 만들어야 합니다. 그런 다음 이 디렉터리 경로를 파이프라인 객체의 메모리 인수로 전달하면 됩니다.

마지막으로, 스크립트나 노트북의 마지막에 rmtree(cache_dir)를 포함시켜 캐시 디렉터리와 그 내용을 제거하세요.

하지만 캐시를 사용할 때 몇 가지 주의할 점이 있습니다(심각한 문제는 아니지만). 자세한 내용은 여기에서 확인하실 수 있습니다.

- https://scikit-learn.org/stable/modules/compose.html#caching-transformers-avoid-repeated-computation

### 7. Inside other objects

파이프라인이 여러개의 transformer를 포함하고 있지만 결국 estimator이다.

In [16]:
isinstance(my_pipe, BaseEstimator)

True

따라서 일반적으로 estimator를 사용할 수 있는 모든 곳에 사용할 수 있다.  
예를 들어 dataleakage를 예방하기 위핸 cross validation에 활용할 수 있다.

```python
from sklearn.model_selection import cross_validate

results = cross_validate(
    estimator=full_pipeline_clf,
    X,
    y,
    cv = 5,
    n_jobs=-1,
    scoring=["accuracy", "logloss"]
)
```

비슷하게 HalbingGridSearch를 같은 것을 사용하려면 다음과 같이 사용하면 된다.  

-https://towardsdatascience.com/11-times-faster-hyperparameter-tuning-with-halvinggridsearch-232ed0160155

```python
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC

# Define the pipeline with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", num_pipe, num_cols),
        ("categorical", cat_pipe, cat_cols),
    ]
)

pipe = Pipeline(
    [("preprocessor", preprocessor), ("classifier", SVC())]
)

param_grid = {
    "preprocessor__numeric__with_mean": [True, False],
    "preprocessor__categorical__min_frequency": [2, 4, 6],
    "classifier__C": [0.1, 1, 10],
    "classifier__kernel": ["linear", "rbf"],
}

search = HalvingGridSearchCV(
    pipe, param_grid, cv=5, factor=2, random_state=42
)
```

StandardScaler의 첫 번째 매개변수인 with_mean은 중첩된 매개변수의 예입니다. 이 매개변수 앞에는 두 개의 지정자, 즉 전처리자와 숫자가 이중 밑줄로 구분되어 있습니다.

중첩된 매개변수는 <단계_이름>__<매개변수> 구문을 따릅니다. 이 경우 with_mean은 두 단계 깊이인 트랜스포머의 매개변수입니다. 내부 파이프라인의 이름은 숫자이고 외부 파이프라인의 이름은 전처리기이므로 preprocessor__numeric__with_mean이 됩니다.

이 구문으로 중첩된 매개변수를 작성하면 모델의 매개변수뿐만 아니라 내부 트랜스포머 자체의 매개변수에 대해서도 최적화할 수 있습니다.

### 8. Custom transformers

FunctionTransformer 클래스를 사용하면 파이프라인에 통합할 수 있는 파이썬 함수를 트랜스포머로 변환할 수 있습니다.   

예를 들어, 데이터 프레임의 각 행에 누락된 값의 수를 나타내는 열을 추가하는 다음 함수를 생각해 보겠습니다:

In [17]:
def num_missing_row(X: pd.DataFrame, y = None):
    
    num_missing = X.isnull().sum(axis = 1)
    
    X["num_missing"] = num_missing
    
    return X

In [18]:
from sklearn.preprocessing import FunctionTransformer

custom_transformer = FunctionTransformer(func = num_missing_row)

numeric_pipe = make_pipeline(
    StandardScaler(),
    custom_transformer,
    LinearRegression()
)
numeric_pipe

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('functiontransformer',
                 FunctionTransformer(func=<function num_missing_row at 0x7f3723cea670>)),
                ('linearregression', LinearRegression())])

- 그외 자세한 내용은 아래를 참고 바란다.
- https://ibexorigin.medium.com/in-depth-guide-to-building-custom-sklearn-transformers-for-any-data-preprocessing-scenario-33450f8b35ff

### 9. Target transformations with a pipeline

대부분의 경우 파이프라인의 트랜스포머는 피처 배열 X에 초점을 맞추지만, 대상 배열 Y에도 일부 전처리가 필요한 경우가 있습니다.

회귀 분석의 일반적인 시나리오는 선형 모델의 적합도를 개선하기 위해 대상을 정규 분포로 만드는 것입니다. 파이프라인 외부에서 정규화를 수행하면 훈련 세트에 데이터 누출이 발생할 가능성이 있습니다.

이 문제를 해결하고 프로세스를 간소화하기 위해 Sklearn은 TransformedTargetRegressor 클래스를 제공합니다. 이 클래스를 사용하면 파이프라인에 직접 대상 배열 변환을 포함시켜 데이터 무결성을 보장하고 상용구 코드를 줄일 수 있습니다.

```python
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer

transformers = ColumnTransformer(...)
full_pipeline = make_pipeline(transformers, LinearRegression())

qt = QuantileTransformer(output_distribution="normal")

tt = TransformedTargetRegressor(
    regressor=full_pipeline, transformer=qt
)

tt.fit(X, y)
```

- TransformedTargetRegressor를 파이프라인 안에 넣어야 하는거 아닌가?  확인요